In [1]:
import sys
sys.path.append('..')

from VarPro import *
from resnet import *
import torchvision
import torchvision.transforms.v2 as transforms
from torchinfo import summary

In [4]:
resnet = ResNet50(in_channels=1, num_classes=10)

In [3]:
summary(resnet, input_size=(128, 3, 32, 32))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [ResNetFeatureModel: 1, Conv2d: 2, BatchNorm2d: 2, Sequential: 2, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Conv2d: 5, BatchNorm2d: 5, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Sequential: 2, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Conv2d: 5, BatchNorm2d: 5, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, Conv2d: 4, BatchNorm2d: 4, 

In [5]:
transform_train = transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32)])

In [6]:
trainset = torchvision.datasets.MNIST(root='./mnist_data',
                                    train=True,
                                    download=True,
                                    transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

In [7]:
inputs, targets = next(iter(train_loader))

In [10]:
resnet.feature_model(inputs).shape

torch.Size([128, 2048])

In [5]:
## Training
lmbda = 1e-1
time_scale = 2**(-10)
lr = 512 * time_scale

# biased or unbiased loss
criterion = VarProCriterion(lmbda, num_classes=10)
                                   
optimizer = torch.optim.SGD(resnet.feature_model.parameters(), lr=lr)
#optimizer.add_param_group({'lr': 5*student_width*lmbda, 'params': student.outer.weight})

In [31]:
inputs, targets = next(iter(train_loader))

In [42]:
inputs.dtype

torch.float32

In [38]:
preds = torch.softmax(10*resnet(inputs), dim=-1)

In [39]:
preds

tensor([[0.1010, 0.1003, 0.1003,  ..., 0.0979, 0.0997, 0.0993],
        [0.1008, 0.1005, 0.1011,  ..., 0.0977, 0.1005, 0.0994],
        [0.1006, 0.1006, 0.0999,  ..., 0.0980, 0.1004, 0.0994],
        ...,
        [0.1011, 0.1005, 0.1000,  ..., 0.0979, 0.1001, 0.0996],
        [0.1003, 0.1006, 0.1003,  ..., 0.0978, 0.0999, 0.1000],
        [0.1003, 0.1007, 0.1003,  ..., 0.0970, 0.1003, 0.0996]],
       grad_fn=<SoftmaxBackward0>)

In [40]:
accuracy(preds, targets)

tensor(0.0938)

In [7]:
features = resnet.feature_model(inputs).clone().detach()
K = features @ features.T / (128 * 512)
torch.linalg.norm(K, ord=2)

tensor(5.2625)

In [8]:
problem = LearningProblem(resnet, train_loader, optimizer, criterion)

In [9]:
epochs = 1
problem.train(epochs, subprogress=True)

log10(loss) = -0.53: 100%|██████████| 59/59 [18:13<00:00, 18.54s/it]


ZeroDivisionError: integer modulo by zero

In [ ]:
inputs, targets = next(iter(train_loader))

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,

In [10]:
l  =[1, 2, 3]

In [15]:
np.array(l).mean().item()

2.0